### <div align="center">***API REQUEST FOR DATABASE***</div>
***
Extracción de los registros relativos a los distritos 7,8,9: **Chamberí**, **Fuencarral-El Pardo**, **Moncloa-Aravaca**.

In [262]:
## CREDENCIALES PARA PODER USAR LA API

api_key= 

### **IMPORTS**

In [263]:
from yelpapi import YelpAPI
import pandas as pd
import requests
import folium
import json

yelp_api = YelpAPI(api_key)

from utils import distritos

## CHAMBERI (6) : FUENCARRAL (7) : MONCLOA (8)
LATITUD = distritos[20]["latitude"]
LONGITUD = distritos[20]["longitude"]
RADIO = 1000

### **EXTRACCION CON LA API**

In [264]:
## CONFIGURACIÓN PARA QUE PANDAS MUESTRE LAS COLUMNAS DEL DF SIN TRUNCARLAS
pd.set_option('display.max_columns', None) 

In [265]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
    'Authorization': 'Bearer ' + api_key
}

res = []
for dis in range(1):
    params = {    
    "latitude": LATITUD,  # Latitud del epicentro de búsqueda
    "longitude": LONGITUD,  # Longitud del epicentro de búsqueda
    "radius": 1500,  # Radio en metros para buscar restaurantes alrededor de la ubicación
    'categories': 'restaurants',  # Seleccionamos la categoría de comercio a extraer
    'limit': 50,  # Número de resultados que recupera (max 50
    }

    response = requests.get(url, headers=headers, params=params) # GET request
    if response.headers.get("Content-Type") == "application/json":
        try:
            # Parse the JSON response
            data = response.json()

            # Extract and process restaurants
            restaurants = data['businesses']        
                
            for restaurant in restaurants:
                description = ""
                id = restaurant['id']
                name = restaurant["name"]
                categories = restaurant['categories']
                rating = restaurant['rating']    
                address = ', '.join(restaurant['location']['display_address'])
                
                ## ALIAS
                if "alias" in restaurant:
                    alias = restaurant["alias"]
                else:
                    alias = None

                ## URL IMAGEN
                if "image_url" in restaurant:
                    image_url = restaurant["image_url"]
                else:
                    image_url = None

                ## URL NEGOCIO
                if "url" in restaurant:
                    url = restaurant["url"]
                else:
                    url = None

                ## CLOSED
                if "is_closed" in restaurant:
                    is_closed = restaurant["is_closed"]
                else:
                    is_closed = None

                ## REVIEW_COUNT
                if "review_count" in restaurant:
                    review_count = restaurant["review_count"]
                else:
                    review_count = None
                
                ## COORDINATES
                if "coordinates" in restaurant:
                    latitude = restaurant["coordinates"]["latitude"]
                    longitude = restaurant["coordinates"]["longitude"]
                else:
                    latitude = None
                    longitude = None
                
                ## LOCATION
                if "location" in restaurant:
                    location = restaurant["location"]
                else:
                    location = None

                ## PHONE
                if "display_phone" in restaurant:
                    display_phone = restaurant["display_phone"]
                else:
                    display_phone = None

                ## PRICE
                if "price" in restaurant:
                    price = restaurant["price"]
                else:
                    price = None 

                ## HOURS   
                if "hours" in restaurant:
                    hours = restaurant["hours"]
                else:
                    hours = None
                
                ## TRANSACTIONS
                if "transactions" in restaurant:
                    transactions = restaurant["transactions"]
                else:
                    transactions = None

                ## LANGUAGES
                if "languages" in restaurant:
                    languages = restaurant["languages"]
                else:
                    languages = None
                
                ## ATRIBUTES
                if "attributes" in restaurant:
                    attributes = restaurant["attributes"]
                else:
                    attributes = None

                ## TRANSIT
                if "transit" in restaurant:
                    transit = restaurant["transit"]
                else:
                    transit = None

                ## BEST TIME
                if "best_time" in restaurant:
                    best_time = restaurant["best_time"]
                else:
                    best_time = None

                ## DELIVERY
                if "delivery" in restaurant:
                    delivery = restaurant["delivery"]
                else:
                    delivery = None

                ## PICKUP
                if "pickup" in restaurant:
                    pickup = restaurant["pickup"]
                else:
                    pickup = None

                ## ONLINE ORDER
                if "online_ordering" in restaurant:
                    online_ordering = restaurant["online_ordering"]
                else:
                    online_ordering = None
                    
                ## IS HOT AND NEW
                if "is_hot_and_new" in restaurant:
                    is_hot_and_new = restaurant["is_hot_and_new"]
                else:
                    is_hot_and_new = None
                        
                ## PRIMEROS 5 REVIEWS 
                for c in categories:
                    description = description + c["title"] + " "
                description = description  

                # Make the API call to retrieve reviews for the current business
                reviews_response = yelp_api.reviews_query(id=restaurant["id"],location="Madrid", lang="es")
                        
                # Extract the reviews from the response
                reviews = reviews_response['reviews']
                        
                # Process and print the reviews
                        
                rev = ""
                for review in reviews[:5]:
                    review_text = review['text']
                    rating = review['rating']
                    reviewer_name = review['user']['name']

                    rev = rev + review_text + ". "   
                        
                
                res.append({
                        "id" : id,
                        "name" : name, 
                        "rating": rating, 
                        "address": address, 
                        "description": description, 
                        "review": rev,
                        "alias":alias,
                        "name":name,
                        "image_url":image_url,
                        "is_closed":is_closed,
                        "review_count":review_count,
                        "latitude":latitude,
                        "longitude": longitude,
                        "location":location,
                        "display_phone":display_phone,
                        "price":price,
                        "hours":hours,
                        "transactions":transactions,
                        "languages":languages,
                        "attributes":attributes,
                        "transit":transit,
                        "best_time":best_time,
                        "delivery":delivery,
                        "pickup":pickup,
                        "online_ordering":online_ordering
                })
        except json.JSONDecodeError:
            print("La respuesta no es un JSON válido")

df_distrito = pd.DataFrame(res)

In [266]:
df_distrito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               50 non-null     object 
 1   name             50 non-null     object 
 2   rating           50 non-null     float64
 3   address          50 non-null     object 
 4   description      50 non-null     object 
 5   review           50 non-null     object 
 6   alias            50 non-null     object 
 7   image_url        50 non-null     object 
 8   is_closed        50 non-null     bool   
 9   review_count     50 non-null     int64  
 10  latitude         50 non-null     float64
 11  longitude        50 non-null     float64
 12  location         50 non-null     object 
 13  display_phone    50 non-null     object 
 14  price            18 non-null     object 
 15  hours            0 non-null      object 
 16  transactions     50 non-null     object 
 17  languages        0

### **CHECK RESULTS**

In [267]:
map_madrid = folium.Map(location=[40.427919,-3.680877], zoom_start=14)

for (index, row) in df_distrito.iterrows():
    folium.Marker(location = [row.loc["latitude"], row.loc["longitude"]],
    #popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid



In [268]:
df_distrito.to_csv("../data/barajas.csv")